In [2]:
import sqlite3
import pandas as pd

In [4]:
conn = sqlite3.connect('finance.db')
cursor = conn.cursor()

In [13]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS companies (
    company_id INTEGER PRIMARY KEY,
    company_name TEXT NOT NULL,
    industry TEXT NOT NULL,
    founded_year INTEGER
)
''')

In [14]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS financial_reports (
    report_id INTEGER PRIMARY KEY,
    company_id INTEGER,
    year INTEGER NOT NULL,
    quarter INTEGER NOT NULL,
    revenue REAL,
    expenses REAL,
    profit REAL,
    FOREIGN KEY (company_id) REFERENCES companies (company_id)
)
''')

In [15]:
companies = [
    (1, 'TechCorp', 'Technology', 2005),
    (2, 'FinBank', 'Finance', 1995),
    (3, 'EcoEnergy', 'Energy', 2010),
    (4, 'HealthPlus', 'Healthcare', 2008),
    (5, 'RetailGiant', 'Retail', 1980)
]

In [16]:
cursor.executemany('''
INSERT OR REPLACE INTO companies (company_id, company_name, industry, founded_year)
VALUES (?, ?, ?, ?)
''', companies)

In [24]:
financial_data = []
for company_id in range(1, 6):  
    for year in range(2020, 2023):  
        for quarter in range(1, 5):  
            revenue = 100000 + company_id * 10000 + year * 1000 + quarter * 100
            expenses = revenue * 0.7 
            profit = revenue - expenses
            
            financial_data.append((
                None,  
                company_id,
                year,
                quarter,
                revenue,
                expenses,
                profit
            ))

In [18]:
cursor.executemany('''
INSERT INTO financial_reports (report_id, company_id, year, quarter, revenue, expenses, profit)
VALUES (?, ?, ?, ?, ?, ?, ?)
''', financial_data)

In [19]:
print("All Companies:")
pd.read_sql_query("SELECT * FROM companies", conn)

All Companies:


,company_id,company_name,industry,founded_year
0,1,TechCorp,Technology,2005
1,2,FinBank,Finance,1995
2,3,EcoEnergy,Energy,2010
3,4,HealthPlus,Healthcare,2008
4,5,RetailGiant,Retail,1980


In [20]:
print("Tech Companies:")
pd.read_sql_query("SELECT * FROM companies WHERE industry = 'Technology'", conn)

Tech Companies:


,company_id,company_name,industry,founded_year
0,1,TechCorp,Technology,2005


In [21]:
print("Recent Financial Reports:")
pd.read_sql_query("SELECT * FROM financial_reports ORDER BY year DESC, quarter DESC LIMIT 10", conn)

Recent Financial Reports:


,report_id,company_id,year,quarter,revenue,expenses,profit
0,12,1,2022,4,2132400.0,1492680.0,639720.0
1,24,2,2022,4,2142400.0,1499680.0,642720.0
2,36,3,2022,4,2152400.0,1506680.0,645720.0
3,48,4,2022,4,2162400.0,1513680.0,648720.0
4,60,5,2022,4,2172400.0,1520680.0,651720.0
5,11,1,2022,3,2132300.0,1492610.0,639690.0
6,23,2,2022,3,2142300.0,1499610.0,642690.0
7,35,3,2022,3,2152300.0,1506610.0,645690.0
8,47,4,2022,3,2162300.0,1513610.0,648690.0
9,59,5,2022,3,2172300.0,1520610.0,651690.0


In [22]:
print("Total Revenue of each company:")
pd.read_sql_query("""
SELECT c.company_name, SUM(f.revenue) as total_revenue
FROM financial_reports f
JOIN companies c ON f.company_id = c.company_id
GROUP BY f.company_id
ORDER BY total_revenue DESC
""", conn)

Total Revenue of each company:


,company_name,total_revenue
0,RetailGiant,26055000.0
1,HealthPlus,25935000.0
2,EcoEnergy,25815000.0
3,FinBank,25695000.0
4,TechCorp,25575000.0


In [23]:
print("2022 Q1 most profitable company:")
pd.read_sql_query("""
SELECT c.company_name, f.profit
FROM financial_reports f
JOIN companies c ON f.company_id = c.company_id
WHERE f.year = 2022 AND f.quarter = 1
ORDER BY f.profit DESC
""", conn)

2022 Q1 most profitable company:


,company_name,profit
0,RetailGiant,651630.0
1,HealthPlus,648630.0
2,EcoEnergy,645630.0
3,FinBank,642630.0
4,TechCorp,639630.0
